In [15]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensorfrom
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np
import os

from scipy.spatial import KDTree

In [16]:
class KDTreeDataset(Dataset):
    def __init__(self, pts_file, split=0):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        # points = np.delete(points, -2, 1)

        if split == 1:
            points = points[:len(points)//2]
        elif split == 2:
            points = points[len(points)//2:]

        print('file points shape', points.shape)
        
        self.dim_scalars = []
        for i in range(4):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
            self.dim_scalars.append(dim_max - dim_min)
        self.data = points

        self.tree = KDTree(self.data[:, :3])
        
    def __len__(self):
        return len(self.data) // 15000
        
    def __getitem__(self, idx):
        # Return batches of 25000 points
        # print('Starting point', self.data[idx * 10000, :3])
        noisy_point = self.data[idx * 15000, :3] + np.random.normal(0, 0.01, 3)[0]
        # print('Noisy point', noisy_point)
        
        _, indicies = self.tree.query(noisy_point, k=25000)

        xyzirn = self.data[indicies, :-1]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[indicies, -1] == 5

        xyzirn = torch.from_numpy(xyzirn.T).float() # intensity, z, y, x from top to bottom? or other way around
        label = torch.tensor(label).long()
        
        # print(xyzirn.size(), label.size())

        return xyzirn, label

class NormalDataset(Dataset):
    def __init__(self, pts_file, split=0):
        points = np.loadtxt(pts_file, delimiter=' ')
        points = np.delete(points, -2, 1)
        points = np.delete(points, -2, 1)
        # points = np.delete(points, -2, 1)

        if split == 1:
            points = points[:len(points)//2]
        elif split == 2:
            points = points[len(points)//2:]

        print(points.shape)
        # do i need to min max intensity, return number, number of returns, etc.? probably not
        self.dim_scalars = []
        for i in range(4):
            dim_min, dim_max = min(points[:,i]), max(points[:,i])
            points[:,i] = (points[:,i] - dim_min) / (dim_max - dim_min)
            self.dim_scalars.append(dim_max - dim_min)
        self.data = points
        
    def __len__(self):
        return len(self.data) // 25000
        
    def __getitem__(self, idx):
        # Return batches of 25000 points
        xyzirn = self.data[idx * 25000: (idx + 1) * 25000, :-1]  # x, y, z, ***intensity, return number, number of returns***
        label = self.data[idx * 25000: (idx + 1) * 25000, -1] == 5

        xyzirn = torch.from_numpy(xyzirn.T).float() # intensity, z, y, x from top to bottom
        label = torch.tensor(label).long()
        
        return xyzirn, label


data_folder = os.path.join(os.getcwd(), r"data\Point Cloud")
training_data = KDTreeDataset(os.path.join(data_folder, r"Traininig.pts"))
validation_data = NormalDataset(os.path.join(data_folder, r"Testing.pts"), split=1)
testing_data = NormalDataset(os.path.join(data_folder, r"Testing.pts"), split=2)
train_dataloader = DataLoader(training_data, batch_size=5, shuffle=True) # can/should i use shuffle, try lowering it?
validation_dataloader = DataLoader(validation_data, batch_size=5, shuffle=False)
test_dataloader = DataLoader(testing_data, batch_size=5, shuffle=False)

num_classes = 2

file points shape (753876, 5)
(205861, 5)
(205861, 5)


In [17]:
print(len(training_data))
print(len(train_dataloader))

50
10


In [18]:
for i, data in enumerate(train_dataloader):
    print(data[0].size())

torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])


torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])
torch.Size([5, 4, 25000])


In [19]:
import open3d as o3d
import numpy as np
import copy
from collections import Counter

def plot_view(entire_view = True):
    # Assuming pc is your point cloud data, in shape (N, 3)
    pc_num = 2

    if entire_view:
        pc = training_data.data[:, :3]
        labels = training_data.data[:, -1] == 5
    else:
        pc, labels = training_data[pc_num]
        pc = pc.T.numpy()[:, :3]
        labels = labels.numpy()

    pc = copy.deepcopy(pc)
    for i in range(3):
        pc[:, i] *= training_data.dim_scalars[i]

    print(Counter(labels), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# plot_view(entire_view = False)

In [20]:
def plot_batch(data):
    pc, labels = data[0].transpose(1,2).reshape(-1, 4).numpy()[:, :3], data[1].view(-1).numpy()
    # print(labels.shape)
    pc = copy.deepcopy(pc)
    for i in range(3):
        pc[:, i] *= training_data.dim_scalars[i]

    print(Counter(list(labels)), len(labels))

    # Define colors for each label
    color_map = {0: [0.5, 0.5, 0.5],  # Gray color for label 0
                1: [1.0, 0.0, 0.0]}  # Red color for label 1

    # Map each label to a color
    colors = np.array([color_map[label] for label in labels])

    # Create point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pc)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([point_cloud])
# for i, data in enumerate(validation_dataloader):
#     print(data[0].size())
#     plot_batch(data)
#     break

In [21]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# device = 'cpu'
print(f"Using {device} device")

Using cuda device


In [22]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1,0,0,0,1,0,0,0,1]).astype(np.float32))).view(1,9).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1,self.k*self.k).repeat(batchsize,1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

class PointNetfeat(nn.Module):
    def __init__(self, global_feat = True, feature_transform = True):
        super(PointNetfeat, self).__init__()
        self.stn = STNkd(k=4)
        self.conv1 = torch.nn.Conv1d(4, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 1024, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.global_feat = global_feat
        self.feature_transform = feature_transform
        if self.feature_transform:
            self.fstn = STNkd(k=64)

    def forward(self, x):
        n_pts = x.size()[2]
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        if self.feature_transform:
            trans_feat = self.fstn(x)
            x = x.transpose(2,1)
            x = torch.bmm(x, trans_feat)
            x = x.transpose(2,1)
        else:
            trans_feat = None

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        if self.global_feat:
            return x, trans, trans_feat
        else:
            x = x.view(-1, 1024, 1).repeat(1, 1, n_pts)
            return torch.cat([x, pointfeat], 1), trans, trans_feat


class PointNetDenseCls(nn.Module):
    def __init__(self, k = 2, feature_transform=True):
        super(PointNetDenseCls, self).__init__()
        self.k = k
        self.feature_transform=feature_transform
        self.feat = PointNetfeat(global_feat=False, feature_transform=feature_transform)
        self.conv1 = torch.nn.Conv1d(1088, 512, 1)
        self.conv2 = torch.nn.Conv1d(512, 256, 1)
        self.conv3 = torch.nn.Conv1d(256, 128, 1)
        self.conv4 = torch.nn.Conv1d(128, self.k, 1)
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.bn3 = nn.BatchNorm1d(128)

    def forward(self, x):
        batchsize = x.size()[0]
        print(x.size())
        n_pts = x.size()[2]
        x, trans, trans_feat = self.feat(x)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.conv4(x)
        x = x.transpose(2,1).contiguous()
        x = F.log_softmax(x.view(-1,self.k), dim=-1)
        x = x.view(batchsize, n_pts, self.k)
        return x, trans_feat
    
# model = PointNetDenseCls(k=2).to(device)

In [23]:
def feature_transform_regularizer(trans):
    d = trans.size(1)
    I = torch.eye(d).unsqueeze(0).to(device)
    loss = torch.linalg.norm(I - torch.bmm(trans, trans.transpose(2,1)), dim=(1,2))
    loss = torch.mean(loss)
    return loss

## Online Code

In [24]:
from sklearn.metrics import f1_score
from collections import Counter


writer = SummaryWriter()
classifier = PointNetDenseCls().to(device)

optimizer = optim.Adam(classifier.parameters(), lr=0.0005, betas=(0.9, 0.999))

num_batch = len(training_data)

for epoch in range(120):
    classifier.train()
    train_loss, train_f1, train_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    for i, data in enumerate(train_dataloader, 1):
        points, target = data
        points, target = points.to(device), target.to(device)
        optimizer.zero_grad()
        pred, trans_feat = classifier(points)
        pred = pred.view(-1, num_classes)
        target = target.view(-1, 1).squeeze()
        print(f'Iteration {i}', pred.size(), target.size())

        loss = F.nll_loss(pred, target)
        loss += feature_transform_regularizer(trans_feat) * 0.001
        loss.backward()
        optimizer.step()
        train_loss += float(loss) # JUST USING LOSS ACCUMULATES HISTORY

        predictions = np.append(predictions, pred.max(1)[1].cpu())
        labels = np.append(labels, target.cpu())

    train_f1 = f1_score(predictions, labels)
    train_acc = sum(predictions == labels)/float(len(labels))
    train_loss /= len(train_dataloader)


    classifier.eval()
    valid_loss, valid_f1, valid_acc = 0.0, 0.0, 0.0
    predictions, labels = np.array([]), np.array([])
    with torch.no_grad():
        for i, data in enumerate(validation_dataloader):
            points, target = data
            points, target = points.to(device), target.to(device)
            pred,  trans_feat = classifier(points)
            pred = pred.view(-1, num_classes)
            target = target.view(-1, 1).squeeze()

            loss = F.nll_loss(pred, target)
            loss += feature_transform_regularizer(trans_feat) * 0.001
            valid_loss += float(loss)

            predictions = np.append(predictions, pred.max(1)[1].cpu())
            labels = np.append(labels, target.cpu())

    valid_f1 = f1_score(predictions, labels)
    valid_acc = sum(predictions == labels)/float(len(labels))
    valid_loss /= len(validation_dataloader)


    writer.add_scalars('losses', {'training':train_loss, 'validation':valid_loss}, global_step=epoch)
    writer.add_scalars('f1 scores', {'training':train_f1, 'validation':valid_f1}, global_step=epoch)

    print(f'[{epoch}] train loss: {train_loss} accuracy: {train_acc} f1 score: {train_f1}')
    print(f'[{epoch}] validation loss: {valid_loss} accuracy: {valid_acc} f1 score: {valid_f1}')
    print()

writer.flush()
writer.close()

torch.Size([5, 4, 25000])
Iteration 1 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 2 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 3 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 4 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 5 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 6 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 7 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 8 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 9 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
Iteration 10 torch.Size([125000, 2]) torch.Size([125000])
torch.Size([5, 4, 25000])
torch.Size([3, 4, 25000])
[0] train loss: 0.7702283978462219 accuracy: 0.669376 f1 score: 0.1741931358600956
[0] validation loss: 0.66231921315

In [ ]:
from sklearn.metrics import f1_score

outputs = None
points = None
x = y = None
def test_PointNet(pointnet, test_dataloader, device):
    global outputs, points, x, y
    pointnet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (points, labels) in enumerate(test_dataloader):
            points, labels = points.to(device), labels.to(device)
            outputs, _ = pointnet(points)
            _, predicted = torch.max(outputs.data, 2)
            total += labels.size(0) * labels.size(1)
            correct += (predicted == labels).sum().item()
            x = labels.view(-1).to('cpu').numpy()
            y = predicted.view(-1).to('cpu').numpy()
            f1 = f1_score(x, y)
            print('F1 score: ', f1)
    
test_PointNet(classifier, test_dataloader, device)

F1 score:  0.3325457767277023
F1 score:  0.34448641403679786


In [ ]:
print(sum(x))
print(sum(y))
print(outputs)

20070
20965
tensor([[[-0.0199, -3.9263],
         [-0.0199, -3.9261],
         [-0.0199, -3.9259],
         ...,
         [-0.3394, -1.2456],
         [-0.1242, -2.1474],
         [-0.1215, -2.1678]],

        [[-0.1984, -1.7152],
         [-0.1861, -1.7731],
         [-0.1867, -1.7700],
         ...,
         [-0.0231, -3.7794],
         [-0.0232, -3.7755],
         [-0.0233, -3.7693]],

        [[-0.0218, -3.8362],
         [-0.0219, -3.8333],
         [-0.0320, -3.4593],
         ...,
         [-1.0450, -0.4334],
         [-1.0428, -0.4346],
         [-1.0892, -0.4102]]], device='cuda:0')


In [ ]:
print(f"Labels has {sum(x == 0)} zeroes and {sum(x == 1)} ones")
print(f"Predictions has {sum(y == 0)} zeroes and {sum(y == 1)} ones")


# writer.add_graph(model, points)
# writer.flush()
# writer.close()

Labels has 54930 zeroes and 20070 ones
Predictions has 54035 zeroes and 20965 ones


73.4029268292683% on roofs (5)
86.77414634146342% on trees (8)